In [1]:
#load lib
import numpy as np
import re, sys, glob
import itertools
from collections import Counter
from keras.preprocessing import text
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from nltk import ngrams
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import keras
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.models import load_model
import math


Using TensorFlow backend.


In [2]:
#load data

'''
Features order in the data look like this 
print STDOUT "S:".$uttranceInfo{"id"}."\t".$wordcount." ".$graphemecount." ".$uttranceInfo{"total_frames"}.
      " ".$uttranceInfo{"avg_loglike"}." ".$uttranceInfo{"total_AMloglike"}." ".
	  $uttranceInfo{"total_LMloglike"}." ".$uttranceInfo{"duration"}." ".
	  $uttranceInfo{"GER"}."\t".$uttranceInfo{"sentence"}."\t".$uttranceInfo{"grapheme"}.
	  "\t".$uttranceInfo{"WER"}."\n";
'''

train="data/dev.mgb2/wer.feats"
test1="data/eval.mgb2/wer.feats"
test2="data/summa_test20170317/wer.feats"


In [3]:
def load_file (file,type='glass'):
    f=open(file,"r")
    lines=f.readlines()
    _id=[]
    _text=[]
    _numerical=[]
    _char=[]
    _ref=[]
    for x in lines:
        _id.append(x.split('\t')[0])
        _text.append(x.split('\t')[2])
        _char.append(x.split('\t')[3])
        _ref.append(x.split('\t')[4])
        if type == 'glass': 
            _numerical.append(x.split('\t')[1])
        else:
            blac_box_numerical=x.split('\t')[1].split(' ')
            numerical2=' '.join(blac_box_numerical[0:2]+blac_box_numerical[6:])
            _numerical.append(numerical2)
        
    f.close()
    return _id, _numerical, _text, _char, _ref

def word_grams(words, min=1, max=4):
    s = []
    for n in range(min, max):
        for ngram in ngrams(words, n):
            s.append(' '.join(str(i) for i in ngram))
    return s

def return_word_char_list (_text,_char):
    _unigram_word_list = [] 
    _trigram_char_list = []
    for i,j in enumerate(_char):
        
        _char_3gram = word_grams (_char[i].split(),1,3)
        for tri in _char_3gram: 
             if tri not in _trigram_char_list: _trigram_char_list.append(tri)
    
        _words_1gram = word_grams (_text[i].split(),1,2)
        for uni in _words_1gram: 
             if uni not in _unigram_word_list: _unigram_word_list.append(uni)
    return _unigram_word_list, _trigram_char_list 
                    
def vectorize (words,chars,word_list,char_list):
    feature_vector = []
    index=0
    
    temp_words = [0] * len(word_list)
    for word in word_grams (words.split(),1,2): 
        for index, word2 in enumerate(word_list): 
            if word == word2:
                #print (index, word2)
                temp_words[index]+=1
                break
    
    temp_chars = [0] * len(char_list)
    for _char in word_grams (chars.split(),1,3): 
        for index, _char2 in enumerate(char_list): 
            if _char == _char2:
                #print (index, _char2)
                temp_chars[index]+=1
                break            
    return temp_words+temp_chars

def make_features (numerical, text, char, word_list, char_list ):
    allfeats= []
    for i, _id in enumerate (text):
        feat = vectorize (text[i],char[i],word_list,char_list)
        allfeats.append (numerical[i].split()+feat)
        if i % 1000 == 0: print ("Processing: ", i)
    nn=np.array(allfeats)
    #nn_minmax = preprocessing.scale(nn)
       
    print (nn.shape)
    return (nn)
    
def get_wc_err (file,dump):
    f=open(file,"r")
    lines=f.readlines()
    _wc=[]
    _err=[]
    _real_wc =[]
    _real_err =[]
    for x in lines:
        err = int(x.split('\t')[4].split(' ')[1])
        wc  = int(x.split('\t')[4].split(' ')[2])
        _real_wc.append (wc)    
        _real_err.append (err)
        
        wc = wc - 2
        if wc < 0: wc =0
        if wc > 20: wc =20
        
        if err > 20: err =20
         
        _wc.append (wc)    
        _err.append (err)
        one_hot_labels_wc = to_categorical(np.array(_wc), num_classes=21)
        one_hot_labels_err = to_categorical(np.array(_err), num_classes=21)
        
    np.savetxt(dump+'.err', np.array(_real_err), fmt='%d')
    np.savetxt(dump+'.wc', np.array(_real_wc), fmt='%d')
        
    return one_hot_labels_wc, one_hot_labels_err


In [4]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

    

def report (test_vec,test_labels,test,ext):
    
    prob_out='prob.out.'+test+'.'+ext
    pred_out='pred.out.'+test+'.'+ext
    class_out='class.out.'+test+'.'+ext
    
    _shift=0
    if ext == "wc" : _shift=2
    #load models
    model = load_model('./my_model.h5')
    
    #save predictions
    pred = model.predict(test_vec)
    np.savetxt(pred_out, pred, delimiter=' ')    
    
    prob = np.copy(pred)
    i=0
    for row in prob: 
      raw = [sigmoid(i) for i in row]
      norm = [float(i)/sum(raw) for i in raw]
      prob[int(i)]=norm
      i=i+1
    
    #save prob
    np.savetxt(prob_out, prob, delimiter=' ',fmt='%f') 
    
    
    test_classes = np.argmax(prob, axis=1)+_shift
    #save classes
    np.savetxt(class_out, test_classes, delimiter=' ',fmt='%d')
    

    test_labels = np.argmax(test_labels, axis=1) # to remove the Keras to_categorical
    
    

    #print confusion matrix 
    confusion_matrix(test_labels, test_classes)

    #print classification report
    print(classification_report(test_labels, test_classes))


    #print prec, recall and f1
    print ("Precision:\t{:0.3f}".format(precision_score(test_labels, test_classes,average='weighted')))
    print ("Recall:  \t{:0.3f}".format(recall_score(test_labels, test_classes,average='weighted')))
    print ("F1 Score:\t{:0.3f}".format(f1_score(test_labels, test_classes,average='weighted')))
    print ("Accuracy:\t{:0.3f}".format(accuracy_score(test_labels, test_classes)))
    


In [5]:
id, numerical, text, char, ref =load_file (train,'glass')

word_list, char_list = return_word_char_list (text, char)
numerical_len=len(numerical[0].split())

feat_len=(len(word_list+char_list)+numerical_len)
print ("Feature dimension: ", feat_len)



train_feats = make_features (numerical, text, char, word_list, char_list )
train_wc, train_err = get_wc_err (train,'train')



id, numerical, text, char, ref =load_file (test1,'glass')
test1_feats = make_features (numerical, text, char, word_list, char_list )
test1_wc, test1_err = get_wc_err (test1,'test1')


id, numerical, text, char, ref =load_file (test2,'glass')
test2_feats = make_features (numerical, text, char, word_list, char_list )
test2_wc, test2_err = get_wc_err (test2,'test2')



Feature dimension:  14964
Processing:  0
Processing:  1000
Processing:  2000
Processing:  3000
Processing:  4000
Processing:  5000
(5842, 14964)
Processing:  0
Processing:  1000
Processing:  2000
Processing:  3000
Processing:  4000
Processing:  5000
(5655, 14964)
Processing:  0
Processing:  1000
(1410, 14964)


In [6]:
#def dnn (feats,labels):
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=feat_len))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(21, activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(feats, labels, epochs=20, batch_size=32)

In [7]:
model.fit(train_feats,train_wc, epochs=30,batch_size=32)
model.save('./my_model.h5', overwrite=True)

report (test1_feats,test1_wc,'test1.glass','wc')
report (test2_feats,test2_wc,'test2.glass','wc')

Epoch 1/30
5842/5842 [==============================] - 38s - loss: 3.0791 - acc: 0.1089    
Epoch 2/30
5842/5842 [==============================] - 38s - loss: 2.5748 - acc: 0.1611    
Epoch 3/30
5842/5842 [==============================] - 38s - loss: 2.4023 - acc: 0.1792    
Epoch 4/30
5842/5842 [==============================] - 38s - loss: 2.2862 - acc: 0.1905    
Epoch 5/30
5842/5842 [==============================] - 38s - loss: 2.1882 - acc: 0.2128    
Epoch 6/30
5842/5842 [==============================] - 38s - loss: 2.1338 - acc: 0.2345    
Epoch 7/30
5842/5842 [==============================] - 37s - loss: 2.0688 - acc: 0.2432    
Epoch 8/30
5842/5842 [==============================] - 38s - loss: 2.0487 - acc: 0.2391    
Epoch 9/30
5842/5842 [==============================] - 37s - loss: 1.9937 - acc: 0.2554    
Epoch 10/30
5842/5842 [==============================] - 37s - loss: 1.9739 - acc: 0.2564    
Epoch 11/30
5842/5842 [==============================] - 37s - loss: 

D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
D:\Users\AMALI\Ana

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        10
          1       0.00      0.00      0.00        21
          2       0.00      0.00      0.00        76
          3       0.00      0.00      0.00       128
          4       0.00      0.00      0.00       219
          5       0.01      0.00      0.01       305
          6       0.02      0.00      0.01       413
          7       0.09      0.15      0.11       462
          8       0.07      0.00      0.00       509
          9       0.11      0.19      0.14       519
         10       0.10      0.06      0.07       509
         11       0.16      0.30      0.20       481
         12       0.23      0.41      0.30       435
         13       0.25      0.25      0.25       363
         14       0.26      0.33      0.29       328
         15       0.22      0.39      0.28       235
         16       0.08      0.01      0.01       185
         17       0.15      0.18      0.16   

In [8]:
model.fit(train_feats,train_err, epochs=30,batch_size=32)
model.save('./my_model.h5', overwrite=True)

report (test1_feats,test1_err,'test1.glass','err')
report (test2_feats,test2_err,'test2.glass','err')

Epoch 1/30
5842/5842 [==============================] - 37s - loss: 3.2201 - acc: 0.1443    
Epoch 2/30
5842/5842 [==============================] - 38s - loss: 2.4695 - acc: 0.1838    
Epoch 3/30
5842/5842 [==============================] - 37s - loss: 2.2456 - acc: 0.1931    
Epoch 4/30
5842/5842 [==============================] - 37s - loss: 2.1754 - acc: 0.2023    
Epoch 5/30
5842/5842 [==============================] - 37s - loss: 2.1591 - acc: 0.2058    
Epoch 6/30
5842/5842 [==============================] - 37s - loss: 2.1333 - acc: 0.2191    
Epoch 7/30
5842/5842 [==============================] - 37s - loss: 2.1111 - acc: 0.2246    
Epoch 8/30
5842/5842 [==============================] - 37s - loss: 2.0762 - acc: 0.2241    
Epoch 9/30
5842/5842 [==============================] - 37s - loss: 2.0664 - acc: 0.2217    
Epoch 10/30
5842/5842 [==============================] - 37s - loss: 2.0468 - acc: 0.2292    
Epoch 11/30
5842/5842 [==============================] - 38s - loss: 

D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.43      0.06      0.10       619
          1       0.34      0.23      0.27       886
          2       0.23      0.57      0.33       868
          3       0.26      0.02      0.04       800
          4       0.16      0.33      0.22       588
          5       0.16      0.18      0.17       498
          6       0.14      0.19      0.16       358
          7       0.12      0.01      0.02       217
          8       0.08      0.01      0.02       208
          9       0.12      0.13      0.13       160
         10       0.00      0.00      0.00       106
         11       0.09      0.02      0.04        89
         12       0.13      0.22      0.17        60
         13       0.00      0.00      0.00        39
         14       0.25      0.10      0.14        31
         15       0.00      0.00      0.00        29
         16       0.06      0.17      0.09        18
         17       0.00      0.00      0.00   

In [9]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt


#rms = sqrt(mean_squared_error(y_actual, y_predicted))

def MAE (txt,glass,black,ref): 
    MAE_glass=mean_absolute_error(glass, ref)
    MAE_black=mean_absolute_error(black, ref)

    RMS_glass = sqrt(mean_squared_error(ref, glass))
    RMS_black = sqrt(mean_squared_error(ref, black))
    
    print  ('MAE:', txt,  "%.2f" % MAE_glass,  "%.2f" % MAE_black)
    print  ('RMSE:', txt,  "%.2f" % RMS_glass,  "%.2f" % RMS_black, '\n')
    

def parse_results(onetwo):
    one_two = str (onetwo)
    print ("Analysis for: ", one_two)
    #wc per sentence
    glass_file_wc = 'class.out.test'+one_two+'.glass.wc'
    black_file_wc = 'class.out.test'+one_two+'.black.wc'
    ref_file_wc   =  'test'+one_two+'.wc'

    glass_np_wc = np.loadtxt(glass_file_wc,usecols=range(0,1),dtype='float32')+2
    black_np_wc = np.loadtxt(black_file_wc,usecols=range(0,1),dtype='float32')+2
    ref_np_wc   = np.loadtxt(ref_file_wc,usecols=range(0,1),dtype='float32')

    MAE ('glass/black: WC :',glass_np_wc,black_np_wc,ref_np_wc)
    
    #err per sentence
    

    
    MAE ('glass/black: ERR :',glass_np_err,black_np_err,ref_np_err)
    

    #wer per sentence
    glass_np_wer=np.divide(glass_np_err,glass_np_wc)*100
    black_np_wer=np.divide(black_np_err,black_np_wc)*100
    ref_np_wer=np.divide(ref_np_err,ref_np_wc)*100

    MAE ('glass/black: WER :',glass_np_wer,black_np_err,ref_np_wer)
    
    
    #Overall WER
    wer_ref   = np.sum(ref_np_err)/np.sum(ref_np_wc)*100
    wer_glass = np.sum(glass_np_err)/np.sum(glass_np_wc)*100
    wer_black = np.sum(black_np_err)/np.sum(black_np_wc)*100
    print ('Overall WER: glass/black:', "%.2f" % wer_ref, "%.2f" % wer_glass, "%.2f" % wer_black)
    print ('###########')

    #dump WER for more analysis
    np.savetxt('wer_ref.'+one_two, ref_np_wer,fmt='%.2f') 
    np.savetxt('wer_black.'+one_two, black_np_wer,fmt='%.2f') 
    np.savetxt('wer_glass.'+one_two, glass_np_wer,fmt='%.2f') 




In [10]:
##Process the black-box features:
id, numerical, text, char, ref =load_file (train,'black')

word_list, char_list = return_word_char_list (text, char)
numerical_len=len(numerical[0].split())

feat_len=(len(word_list+char_list)+numerical_len)
print ("Feature dimension: ", feat_len)



train_feats = make_features (numerical, text, char, word_list, char_list )
train_wc, train_err = get_wc_err (train,'train')



id, numerical, text, char, ref =load_file (test1,'black')
test1_feats = make_features (numerical, text, char, word_list, char_list )
test1_wc, test1_err = get_wc_err (test1,'test1')


id, numerical, text, char, ref =load_file (test2,'black')
test2_feats = make_features (numerical, text, char, word_list, char_list )
test2_wc, test2_err = get_wc_err (test2,'test2')



Feature dimension:  14960
Processing:  0
Processing:  1000
Processing:  2000
Processing:  3000
Processing:  4000
Processing:  5000
(5842, 14960)
Processing:  0
Processing:  1000
Processing:  2000
Processing:  3000
Processing:  4000
Processing:  5000
(5655, 14960)
Processing:  0
Processing:  1000
(1410, 14960)


In [11]:
#def dnn (feats,labels):
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=feat_len))
model.add(Dropout(0.2))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(21, activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
#model.fit(feats, labels, epochs=20, batch_size=32)

In [12]:
model.fit(train_feats,train_wc, epochs=30,batch_size=32)
model.save('./my_model.h5', overwrite=True)

report (test1_feats,test1_wc,'test1.black','wc')
report (test2_feats,test2_wc,'test2.black','wc')

Epoch 1/30
5842/5842 [==============================] - 38s - loss: 2.6431 - acc: 0.1511    
Epoch 2/30
5842/5842 [==============================] - 38s - loss: 2.2223 - acc: 0.2126    
Epoch 3/30
5842/5842 [==============================] - 37s - loss: 2.0167 - acc: 0.2629    
Epoch 4/30
5842/5842 [==============================] - 38s - loss: 1.8991 - acc: 0.2996    
Epoch 5/30
5842/5842 [==============================] - 38s - loss: 1.8048 - acc: 0.3242    
Epoch 6/30
5842/5842 [==============================] - 38s - loss: 1.7305 - acc: 0.3521    
Epoch 7/30
5842/5842 [==============================] - 38s - loss: 1.6347 - acc: 0.3925    
Epoch 8/30
5842/5842 [==============================] - 38s - loss: 1.5627 - acc: 0.4249    
Epoch 9/30
5842/5842 [==============================] - 38s - loss: 1.4861 - acc: 0.4575    
Epoch 10/30
5842/5842 [==============================] - 38s - loss: 1.4062 - acc: 0.4964    
Epoch 11/30
5842/5842 [==============================] - 38s - loss: 

D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
D:\Users\AMALI\Ana

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        10
          1       0.00      0.00      0.00        21
          2       0.00      0.00      0.00        76
          3       0.00      0.00      0.00       128
          4       0.09      0.04      0.05       219
          5       0.11      0.07      0.08       305
          6       0.14      0.09      0.11       413
          7       0.13      0.06      0.08       462
          8       0.11      0.10      0.11       509
          9       0.17      0.06      0.08       519
         10       0.15      0.31      0.20       509
         11       0.12      0.07      0.09       481
         12       0.12      0.15      0.13       435
         13       0.12      0.13      0.12       363
         14       0.09      0.14      0.11       328
         15       0.10      0.09      0.09       235
         16       0.09      0.11      0.10       185
         17       0.09      0.22      0.13   

In [13]:
model.fit(train_feats,train_err, epochs=30,batch_size=32)
model.save('./my_model.h5', overwrite=True)

report (test1_feats,test1_err,'test1.black','err')
report (test2_feats,test2_err,'test2.black','err')

Epoch 1/30
5842/5842 [==============================] - 38s - loss: 3.1903 - acc: 0.1559    
Epoch 2/30
5842/5842 [==============================] - 38s - loss: 2.4076 - acc: 0.1801    
Epoch 3/30
5842/5842 [==============================] - 41s - loss: 2.2463 - acc: 0.1960    
Epoch 4/30
5842/5842 [==============================] - 39s - loss: 2.1657 - acc: 0.2010    
Epoch 5/30
5842/5842 [==============================] - 38s - loss: 2.1208 - acc: 0.2203    
Epoch 6/30
5842/5842 [==============================] - 38s - loss: 2.0983 - acc: 0.2095    
Epoch 7/30
5842/5842 [==============================] - 38s - loss: 2.0677 - acc: 0.2200    
Epoch 8/30
5842/5842 [==============================] - 38s - loss: 2.0460 - acc: 0.2402    
Epoch 9/30
5842/5842 [==============================] - 38s - loss: 2.0118 - acc: 0.2446    
Epoch 10/30
5842/5842 [==============================] - 38s - loss: 1.9944 - acc: 0.2506    
Epoch 11/30
5842/5842 [==============================] - 38s - loss: 

D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
D:\Users\AMALI\Anaconda3\envs\ztdl\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0       0.33      0.10      0.16       619
          1       0.27      0.14      0.18       886
          2       0.21      0.26      0.24       868
          3       0.20      0.23      0.21       800
          4       0.15      0.32      0.20       588
          5       0.13      0.04      0.07       498
          6       0.11      0.22      0.15       358
          7       0.09      0.05      0.06       217
          8       0.14      0.06      0.09       208
          9       0.12      0.16      0.14       160
         10       0.00      0.00      0.00       106
         11       0.14      0.26      0.18        89
         12       0.17      0.03      0.06        60
         13       0.02      0.03      0.02        39
         14       0.07      0.10      0.08        31
         15       0.00      0.00      0.00        29
         16       0.08      0.44      0.13        18
         17       0.18      0.18      0.18   

In [14]:
#summary
parse_results (1)
print ('\n')
parse_results (2)


Analysis for:  1
MAE: glass/black: WC : 1.69 2.21
RMSE: glass/black: WC : 2.12 2.69 

MAE: glass/black: ERR : 1.61 1.87
RMSE: glass/black: ERR : 2.21 2.49 

MAE: glass/black: WER : 13.85 28.08
RMSE: glass/black: WER : 18.27 35.76 

Overall WER: glass/black: 33.03 27.84 31.63
###########


Analysis for:  2
MAE: glass/black: WC : 1.65 2.18
RMSE: glass/black: WC : 2.09 2.70 

MAE: glass/black: ERR : 1.73 2.05
RMSE: glass/black: ERR : 2.42 2.78 

MAE: glass/black: WER : 12.72 24.32
RMSE: glass/black: WER : 17.04 34.56 

Overall WER: glass/black: 28.51 24.80 29.32
###########
